In [32]:
import csv
from collections import Counter, defaultdict
import scipy.special

In [9]:
sample_to_sex, sample_to_role, sample_to_asd = dict(), dict(), dict()
family_to_individuals = defaultdict(list)

with open('../phenotypes/spark/individuals.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    ind_index = header.index('subject_sp_id')
    family_index = header.index('family_id')
    sex_index = header.index('sex')
    asd_index = header.index('asd')
    role_index = header.index('role')
    
    for pieces  in reader:
        ind_id = pieces[ind_index]
        sample_to_sex[ind_id] = '2' if pieces[sex_index]=='Female' else '1' if pieces[sex_index]=='Male' else '-1'
        sample_to_role[ind_id] = pieces[role_index]
        sample_to_asd[ind_id] = '2' if pieces[asd_index]=='TRUE' else '1' if pieces[asd_index]=='FALSE' else '-1'
        family_to_individuals[pieces[family_index]].append(ind_id)


In [27]:
import json
with open('../../DATA/spark/genotypes/samples.json', 'r') as f:
    spark_exome_samples = set(json.load(f))

In [34]:
no_mother, no_father, multiple_mothers, multiple_fathers = 0, 0, 0, 0
no_parents = 0
parents_ok = 0

no_mother_sibpairs, no_father_sibpairs, no_parents_sibpairs, parents_ok_sibpairs = 0, 0, 0, 0
with open('../../DATA/spark/spark.ped', 'w+') as f:
    for fam, inds in family_to_individuals.items():
        mothers = [x for x in inds if sample_to_role[x]=='Mother']
        fathers = [x for x in inds if sample_to_role[x]=='Father']
        children = [x for x in inds if sample_to_role[x]=='Proband' or sample_to_role[x]=='Sibling']

        if len(mothers)==0 and len(fathers)==0:
            no_parents += len(children)
            no_parents_sibpairs += scipy.special.comb(len(children), 2)
            mother_id, father_id = '0', '0'
        else:
            if len(mothers)==0:
                no_mother += len(children)
                no_mother_sibpairs += scipy.special.comb(len(children), 2)
                mother_id = '0'
            elif len(mothers)==1:
                mother_id = mothers[0]
            else:
                multiple_mothers += len(children)
                mother_id = '0'

            if len(fathers)==0:
                no_father += len(children)
                no_father_sibpairs += scipy.special.comb(len(children), 2)
                father_id = '0'
            elif len(fathers)==1:
                father_id = fathers[0]
            else:
                multiple_fathers += len(children)
                father_id = '0'
                    
        if mother_id != '0' and father_id != '0':
            parents_ok += len(children)
            parents_ok_sibpairs += scipy.special.comb(len(children), 2)
                
        for child in children:
            f.write('\t'.join([fam, child, father_id, mother_id, sample_to_sex[child], sample_to_asd[child]]) + '\n')
            
print('parents ok', parents_ok, 'no parents', no_parents, 'no mother', no_mother, 'no father', no_father, 'multiple mothers', multiple_mothers, 'multiple fathers', multiple_fathers )
print('parents ok', parents_ok_sibpairs, 'no parents', no_parents_sibpairs, 'no mother', no_mother_sibpairs, 'no father', no_father_sibpairs )

# no parents 39 no mother 187 no father 2442 multiple mothers 0 multiple fathers 0          

parents ok 32478 no parents 4556 no mother 4225 no father 40096 multiple mothers 0 multiple fathers 0
parents ok 15043.0 no parents 703.0 no mother 1579.0 no father 16170.0
